In [93]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [94]:
import pandas as pd
import numpy as np
import os

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.util import ngrams

from wordcloud import WordCloud

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.metrics import classification_report,confusion_matrix

from collections import defaultdict
from collections import Counter
plt.style.use('ggplot')
stop=set(stopwords.words('english'))

import re
from nltk.tokenize import word_tokenize
import gensim
import string

from tqdm import tqdm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM,Dense, SpatialDropout1D, Dropout
from keras.initializers import Constant
from keras.optimizers import Adam

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [95]:
#mejor
#without Dropout
Adam(lr=6e-6)
validation_split = 0.2
epochs = 3
batch_size = 16
target_corrected = False

In [96]:
#anterior probar
random_state_split = 2
Dropout_num = 0
learning_rate = 6e-6
valid = 0.2
epochs_num = 3
batch_size_num = 16
target_corrected = False
target_big_corrected = False

In [97]:
tweet= pd.read_csv('/content/drive/My Drive/nlp-getting-started/train.csv')
test=pd.read_csv('/content/drive/My Drive/nlp-getting-started/test.csv')
submission = pd.read_csv("/content/drive/My Drive/nlp-getting-started/sample_submission.csv")

In [98]:
ids_with_target_error = [328,443,513,2619,3640,3900,4342,5781,6552,6554,6570,6701,6702,6729,6861,7226]
tweet.loc[tweet['id'].isin(ids_with_target_error),'target'] = 0

In [99]:
tweet.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [100]:
# extracting the number of examples of each class
Real_len = tweet[tweet['target'] == 1].shape[0]
Not_len = tweet[tweet['target'] == 0].shape[0]

In [101]:
def length(text):
    #Funcion que devuelve el largo del texto
    return len(text)

In [102]:
tweet['length'] = tweet['text'].apply(length)

In [105]:
def create_corpus(target):
    lista=[] #Creo lista vacia
    
    for x in tweet[tweet['target']==target]['text'].str.split():
        for i in x:
            lista.append(i)
    return lista

In [106]:
def split_df(tweet, target):
    lista=[] #Creo lista vacia
    
    for x in tweet[tweet['target']==target]['text'].str.split():
        for i in x:
            lista.append(i)
    return lista

In [107]:
#StopWords
np.array(stop)

array({"you'll", 'itself', 'before', 'doing', 'now', 'nor', 'than', 'their', 'if', 'theirs', 'then', 'more', 'yourself', 're', 'you', 'wouldn', 'her', 'which', "you've", 'ours', 'it', 'has', 'mustn', 'only', 'shan', "you're", 'herself', 'can', 'been', 'both', 'yours', 'between', 'own', 'aren', 'very', 'hadn', 'and', 'haven', 'about', 'them', 'y', 'not', 'each', "didn't", 'until', 'during', 'there', 'be', 'his', 'him', 'down', 'who', 's', 'hasn', 'are', 'below', 't', 'off', 'further', 'so', "couldn't", 'such', 'this', 'again', 'all', 'she', 'don', "hasn't", 'is', 'me', 'should', 'your', 'were', 'but', "aren't", 'weren', 'these', 'd', 'ain', 'here', 'couldn', 'does', 'a', "you'd", 'wasn', 'ourselves', 'am', 'while', 'from', 'where', 'to', 'my', "it's", 'how', 'any', 'did', 'of', "don't", 'through', 'over', 'once', 'hers', 'above', 'being', 'do', 'with', 'or', 'mightn', 've', "mustn't", "shouldn't", "won't", 'yourselves', 'same', 'when', 'other', 'at', "doesn't", 'll', 'those', 'we', 'hav

In [108]:
def get_top_tweet_bigrams(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 2)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [109]:
df=pd.concat([tweet,test])
df.shape

(10876, 6)

In [110]:
df.head(5)

,id,keyword,location,text,target,length
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1.0,69.0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1.0,38.0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1.0,133.0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1.0,65.0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1.0,88.0


In [111]:
#Borrar URLs
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

In [112]:
#Borra formatos HTML
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

In [113]:
#Remover Emojis
#Fuente: https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [114]:
def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

In [115]:
df['text']=df['text'].apply(lambda x : remove_URL(x))
df['text']=df['text'].apply(lambda x : remove_html(x))
df['text']=df['text'].apply(lambda x: remove_emoji(x))
df['text']=df['text'].apply(lambda x : remove_punct(x))

In [116]:
palabras_con_1 = split_df(df,1)
len(palabras)


46034

In [117]:
corpus_new1[:10]

['Our',
 'Deeds',
 'are',
 'the',
 'Reason',
 'of',
 'this',
 'earthquake',
 'May',
 'ALLAH']

In [118]:
palabras_con_0 = split_df(df,0)
len(corpus_new0)

60713

In [119]:
corpus_new0[:10]

['Whats', 'up', 'man', 'I', 'love', 'fruits', 'Summer', 'is', 'lovely', 'My']

In [120]:
df.head(10)

,id,keyword,location,text,target,length
0,1,NaN,NaN,Our Deeds are the Reason of this earthquake Ma...,1.0,69.0
1,4,NaN,NaN,Forest fire near La Ronge Sask Canada,1.0,38.0
2,5,NaN,NaN,All residents asked to shelter in place are be...,1.0,133.0
3,6,NaN,NaN,13000 people receive wildfires evacuation orde...,1.0,65.0
4,7,NaN,NaN,Just got sent this photo from Ruby Alaska as s...,1.0,88.0
5,8,NaN,NaN,RockyFire Update California Hwy 20 closed in ...,1.0,110.0
6,10,NaN,NaN,flood disaster Heavy rain causes flash floodin...,1.0,95.0
7,13,NaN,NaN,Im on top of the hill and I can see a fire in ...,1.0,59.0
8,14,NaN,NaN,Theres an emergency evacuation happening now i...,1.0,79.0
9,15,NaN,NaN,Im afraid that the tornado is coming to our area,1.0,52.0


In [121]:
def list_token(df):
    lista=[]
    for tweet in tqdm(df['text']):
        words=[word.lower() for word in word_tokenize(tweet)]
        lista.append(words)
    return lista   

In [122]:
corpus = list_token(df)

100%|██████████| 10876/10876 [00:01<00:00, 7384.41it/s]


In [123]:
embedding_dict={}
with open('/content/drive/My Drive/glove.twitter.27B.200d.txt','r') as f:
    for line in f:
        values=line.split()
        word = values[0]
        vectors=np.asarray(values[1:],'float32')
        embedding_dict[word]=vectors
f.close()

In [124]:
MAX_LEN=50
tokenizer_obj=Tokenizer()
tokenizer_obj.fit_on_texts(corpus)
sequences=tokenizer_obj.texts_to_sequences(corpus)

tweet_pad=pad_sequences(sequences,maxlen=MAX_LEN,truncating='post',padding='post')

In [125]:
word_index=tokenizer_obj.word_index
print('Number of unique words:',len(word_index))

Number of unique words: 22807


In [127]:
num_words=len(word_index)+1
embedding_matrix=np.zeros((num_words,200))

for word,i in tqdm(word_index.items()):
    if i < num_words:
        emb_vec=embedding_dict.get(word)
        if emb_vec is not None:
            embedding_matrix[i]=emb_vec 

100%|██████████| 22807/22807 [00:00<00:00, 332178.69it/s]


In [128]:
tweet_pad[0][0:]

array([ 100, 5778,   20,    1,  814,    5,   18,  237,  136, 1902, 3703,
         68,   39,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0], dtype=int32)

In [131]:
model=Sequential()

embedding=Embedding(num_words,200,embeddings_initializer=Constant(embedding_matrix),
                   input_length=MAX_LEN,trainable=False)

model.add(embedding)
model.add(SpatialDropout1D(0.2))
model.add(LSTM(200, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))


optimzer=Adam(learning_rate=3e-4)

model.compile(loss='binary_crossentropy',optimizer=optimzer,metrics=['accuracy'])

In [132]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 50, 200)           4561600   
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 50, 200)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 200)               320800    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
Total params: 4,882,601
Trainable params: 321,001
Non-trainable params: 4,561,600
_________________________________________________________________


In [133]:
train=tweet_pad[:tweet.shape[0]]
test=tweet_pad[tweet.shape[0]:]

In [134]:
X_train,X_test,y_train,y_test=train_test_split(train,tweet['target'].values,test_size=0.2)
print('Shape of train',X_train.shape)
print("Shape of Validation ",X_test.shape)

Shape of train (6090, 50)
Shape of Validation  (1523, 50)


In [135]:
history=model.fit(X_train,y_train,batch_size=4,epochs=15,validation_data=(X_test,y_test),verbose=2)

Train on 6090 samples, validate on 1523 samples
Epoch 1/15
 - 118s - loss: 0.5690 - accuracy: 0.7351 - val_loss: 0.5060 - val_accuracy: 0.7781
Epoch 2/15
 - 119s - loss: 0.5290 - accuracy: 0.7616 - val_loss: 0.5015 - val_accuracy: 0.7814
Epoch 3/15
 - 117s - loss: 0.5147 - accuracy: 0.7667 - val_loss: 0.4389 - val_accuracy: 0.8096
Epoch 4/15
 - 116s - loss: 0.4938 - accuracy: 0.7755 - val_loss: 0.4430 - val_accuracy: 0.8122
Epoch 5/15
 - 118s - loss: 0.5112 - accuracy: 0.7793 - val_loss: 0.4588 - val_accuracy: 0.7735
Epoch 6/15
 - 116s - loss: 0.5199 - accuracy: 0.7614 - val_loss: 0.5665 - val_accuracy: 0.5686
Epoch 7/15
 - 117s - loss: 0.5864 - accuracy: 0.6785 - val_loss: 0.4650 - val_accuracy: 0.8004
Epoch 8/15
 - 116s - loss: 0.4930 - accuracy: 0.7754 - val_loss: 0.4298 - val_accuracy: 0.8194
Epoch 9/15
 - 118s - loss: 0.4711 - accuracy: 0.7829 - val_loss: 0.4309 - val_accuracy: 0.8109
Epoch 10/15
 - 115s - loss: 0.4666 - accuracy: 0.7924 - val_loss: 0.4677 - val_accuracy: 0.7984
E

In [136]:
train_pred_GloVe = model.predict(train)
train_pred_GloVe_int = train_pred_GloVe.round().astype('int')

In [137]:
test_pred_GloVe = model.predict(test)
test_pred_GloVe_int = test_pred_GloVe.round().astype('int')

In [138]:
submission2 = submission

In [139]:
submission2['target'] = test_pred_GloVe_int
submission2.head(10)

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
5,12,0
6,21,0
7,22,0
8,27,0
9,29,0


In [140]:
submission2.to_csv("/content/drive/My Drive/nlp-getting-started/submission200d15epoch.csv", index=False, header=True)